In [1]:
import sys
# comprobar si pymongo está instalado, y hacerlo en otro caso
try:
    import pymongo
    print("pymongo está en el sistema!")
except ImportError as e:
    !{sys.executable} -m pip install --upgrade --user pymongo
    import pymongo

try:
    import pprint
    print("pprint está en el sistema!")
except ImportError as e:
    !{sys.executable} -m pip install --upgrade --user pprint
    import pprint

from pprint import pprint # para mostrar los json bonitos
from pymongo import MongoClient


pymongo está en el sistema!
pprint está en el sistema!


In [2]:
# conexión a mongo
servidor = 'localhost'
puerto = 27017

# creara una colección tweet y otra user
basededatos = 'mirko'

client = MongoClient(servidor, puerto)
db = client[basededatos]

In [3]:
# vamos a eliminar tweets que tengan un campo direccion que no parezca de un domicilio partiular
import pandas as pd

cursor=db.tweet.find()
print("Eliminamos tweets cuya dirección no parezca un domicilio particular")

#primero elimino los que no sean casas:

nocasa = ["tourism","industrial","amenity","building","highway","historic","shop","leisure","office"]
borradosfiltro1=0

for doc in cursor:
    if doc is not None and "direccion" in doc:
        d = doc["direccion"]["address"]
        if any([no in d for no in nocasa]):
            db.tweet.delete_one({"_id":doc["_id"]})
            borradosfiltro1 +=1
print("Se han borrado",borradosfiltro1,"tweets")

Eliminamos tweets cuya dirección no parezca un domicilio particular
Se han borrado 0 tweets


In [4]:
!pip install --upgrade statistics

In [5]:
from collections import Counter

# quitar las direcciones antiguas
db.user.update_many({},{"$unset":{"direccion":True}})
# ahora tenemos que añadir a cada usuario un campo direccion
cursor = db.user.find()
borrados=0
for doc in cursor:
    id = doc["_id"]
    cursor_tweet = db.tweet.find({"userid":id})
    #numero de tweets de este usuario
    total=0
    #creo un array de direcciones para cada usuario para hallar la moda del array y quedarme así con la dir más recurrente
    dirsuser=[]
    for tweet in cursor_tweet:
        dir = tweet["direccion"]
        #address = dir["display_name"]
        dirsuser.append(dir)
        total+=1
    if(len(dirsuser)==0):
        #si no tiene tweets porque los he borrado en la anterior fase, entonces lo elimino
        db.user.delete_one({"_id":doc["_id"]})
        borrados +=1
    else:  
        #print(dirsuser)
        #calculo la moda del array diruser
        c = Counter([dir["display_name"] for dir in dirsuser])
        mc = c.most_common(1)[0]
        if mc[1]>len(dirsuser)//2:
            dn = mc[0]
            #print(mc, len(dirsuser))
            dir_elegida = [dir for dir in dirsuser if dir["display_name"]==dn][0]
            #print(dir_elegida)
            db.user.update_one({"_id":id},{"$set":{"direccion":dir_elegida}})
        #else:
            #print("No aceptado")
    
print("se han borrado ",borrados," usuarios")

se han borrado  0  usuarios


In [6]:
# borramos los que han quedado sin clave direccion porque no había una moda clara
res = db.user.delete_many({"direccion":{"$exists":0}})
print(res.deleted_count, "borrados")

0 borrados


In [7]:
!mongoexport --db=mirko --collection=tweet  --out=tweet.json
!mongoexport --db=mirko --collection=user  --out=user.json

2021-11-08T21:29:54.279+0100	connected to: mongodb://localhost/
2021-11-08T21:29:54.553+0100	exported 3170 records
2021-11-08T21:29:55.241+0100	connected to: mongodb://localhost/
2021-11-08T21:29:55.378+0100	exported 1698 records


Dos ficheros, uno direcciones asociadas a secciones, y el otro secciones y renta per capita de la seccion